In [2]:
import cv2
import numpy as np
from datetime import datetime
import os

def log_finish_time(task_name):
    """
    Log the finish time of a task to the console.
    """
    finish_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"{task_name} completed at: {finish_time}")

def resize_and_pad_single_folder(input_folder, output_folder, target_width, target_height):
    """
    Resize landscape images to a target width and pad height to reach the target height.
    Handles both .jpg and .png files.
    Converts all images to JPEG format and saves them with 95 quality.
    """
    print(f"Processing folder: {input_folder}")
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    for file_name in os.listdir(input_folder):
        if file_name.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_folder, file_name)
            output_file_name = os.path.splitext(file_name)[0] + ".jpg"  # Convert all to JPEG
            output_path = os.path.join(output_folder, output_file_name)

            # Read the image
            image = cv2.imread(input_path)
            if image is None:
                print(f"Error reading {input_path}. Skipping.")
                continue

            original_height, original_width = image.shape[:2]

            # Resize the image to target width, keeping aspect ratio
            scale_ratio = target_width / original_width
            new_height = int(original_height * scale_ratio)
            resized_image = cv2.resize(image, (target_width, new_height))

            # Add padding to top and bottom to reach target height
            padding_needed = target_height - new_height
            if padding_needed > 0:
                top_padding = padding_needed // 2
                bottom_padding = padding_needed - top_padding
                padded_image = cv2.copyMakeBorder(
                    resized_image, top_padding, bottom_padding, 0, 0,
                    borderType=cv2.BORDER_CONSTANT, value=[0, 0, 0]  # Black padding
                )
            else:
                padded_image = resized_image

            # Save the processed image as JPEG
            cv2.imwrite(output_path, padded_image, [cv2.IMWRITE_JPEG_QUALITY, 95])
            print(f"Resized and padded: {output_path}")

    print("Completed resizing and padding.")
    log_finish_time("Resizing and Padding")

def adjust_annotations_single_folder(image_folder, annotation_folder, output_annotation_folder, target_width, target_height):
    """
    Adjust YOLO annotations for landscape images and save them for a batch test case.
    """
    print(f"Adjusting annotations in folder: {annotation_folder}")

    if not os.path.exists(output_annotation_folder):
        os.makedirs(output_annotation_folder)

    for annotation_file in os.listdir(annotation_folder):
        if annotation_file.endswith('.txt'):
            annotation_path = os.path.join(annotation_folder, annotation_file)
            image_path = os.path.join(image_folder, annotation_file.replace('.txt', '.jpg'))
            output_annotation_path = os.path.join(output_annotation_folder, annotation_file)

            # Check if the corresponding image exists
            if not os.path.exists(image_path):
                print(f"Image file not found for annotation: {annotation_file}. Skipping.")
                continue

            # Get image dimensions
            image = cv2.imread(image_path)
            original_height, original_width = image.shape[:2]

            padding_needed = target_height - int(original_height * (target_width / original_width))
            top_padding = padding_needed // 2

            adjusted_lines = []

            # Read annotations
            with open(annotation_path, 'r') as file:
                annotations = file.readlines()

            for line in annotations:
                class_id, x_center, y_center, width, height = map(float, line.strip().split())

                # Scale coordinates and dimensions
                x_center_scaled = x_center * original_width
                y_center_scaled = y_center * original_height
                width_scaled = width * original_width
                height_scaled = height * original_height

                # Apply padding offsets
                y_center_scaled += top_padding

                # Normalize back to target dimensions
                x_center_normalized = x_center_scaled / target_width
                y_center_normalized = y_center_scaled / target_height
                width_normalized = width_scaled / target_width
                height_normalized = height_scaled / target_height

                # Append adjusted annotation
                adjusted_lines.append(
                    f"{int(class_id)} {x_center_normalized:.6f} {y_center_normalized:.6f} {width_normalized:.6f} {height_normalized:.6f}"
                )

            # Save adjusted annotations
            with open(output_annotation_path, 'w') as file:
                file.write('\n'.join(adjusted_lines))

    print("Completed annotation adjustment.")
    log_finish_time("Annotation Adjustment")

# Paths and target dimensions
parent_input_folder = "D:/FlagDetectionDatasets/ExportedDatasetsReducedML"
target_width = 1920
target_height = 1080

# only "Job_166"
input_folder = os.path.join(parent_input_folder, "Job_166")
output_folder = os.path.join(parent_input_folder, "Job_166_r")
annotation_folder = input_folder  
output_annotation_folder = output_folder  

# Resize and pad images
resize_and_pad_single_folder(input_folder, output_folder, target_width, target_height)

# Adjust annotations
adjust_annotations_single_folder(output_folder, annotation_folder, output_annotation_folder, target_width, target_height)


Processing folder: D:/FlagDetectionDatasets/ExportedDatasetsReducedML\Job_166
Resized and padded: D:/FlagDetectionDatasets/ExportedDatasetsReducedML\Job_166_r\Job_166000007.jpg
Resized and padded: D:/FlagDetectionDatasets/ExportedDatasetsReducedML\Job_166_r\Job_166000008.jpg
Resized and padded: D:/FlagDetectionDatasets/ExportedDatasetsReducedML\Job_166_r\Job_166000009.jpg
Resized and padded: D:/FlagDetectionDatasets/ExportedDatasetsReducedML\Job_166_r\Job_166000006.jpg
Resized and padded: D:/FlagDetectionDatasets/ExportedDatasetsReducedML\Job_166_r\Job_166000011.jpg
Resized and padded: D:/FlagDetectionDatasets/ExportedDatasetsReducedML\Job_166_r\Job_166000012.jpg
Resized and padded: D:/FlagDetectionDatasets/ExportedDatasetsReducedML\Job_166_r\Job_166000014.jpg
Resized and padded: D:/FlagDetectionDatasets/ExportedDatasetsReducedML\Job_166_r\Job_166000020.jpg
Resized and padded: D:/FlagDetectionDatasets/ExportedDatasetsReducedML\Job_166_r\Job_166000023.jpg
Resized and padded: D:/FlagDete